In [29]:
import pandas as pd
import numpy as np
from random import gauss
from math import exp, sqrt
import math
import datetime
from numpy import *
import matplotlib.pyplot as plt
from WindPy import w
from arch import arch_model
import scipy.stats as si
w.start()

.ErrorCode=0
.Data=[Already conntected!]

In [30]:
def getDatesList(beginningDate, endingDate):
    """
    获取交易日序列，return a list of dates

    参数：
        起始日期，终止日期
        
    返回值：
        这段日期的交易日，格式为list, 每个日期格式为datetime.datetime
    """
    #从wind获取datetime列表
    w.isconnected()
    
    #日期list
    datesListInDatetime = w.tdays(beginningDate, endingDate, "").Data[0]
    return datesListInDatetime

In [31]:
def getStockData(startDate, endDate, stockCode):
    """
    获取某个指数某一段时间的收盘价格，返回值为WindData

    参数：startDate：起始日期，字符串格式
           endDate：终止日期，字符串格式
         benchmark：需要查询的指数或者股票代码，字符串格式

    返回值：WindData，e.g.
          .ErrorCode=0
          .Codes=[000905.SH]
          .Fields=[CLOSE]
          .Times=[20201201,20201202,20201203,20201204,20201207,20201208,20201209,20201210,20201211]
          .Data=[[6426.6155,6434.9791,6440.8808,6464.7345,6424.6823,6423.6513,6307.2207,6320.0956,6209.427]]
    """
    w.isconnected()
    closingData = w.wsd(stockCode, "close", startDate, endDate, "")
    return closingData

In [32]:
def calculateStockPriceVolatility(stockCode, startDate, endDate, tradingDaysPerYear):
    """
    计算标的在一段时间内的年化波动率
    stockCode: 标的代码
    startDate: 开始日期
    endDate: 结束日期
    tradingDaysPerYear: 一年交易日天数
    """
    closingPrice = getStockData(startDate, endDate, stockCode).Data[0]
    
    dailyReturn = [0]
    continuousCompoundReturn = [0]
    for i in range(1, len(closingPrice)):
        dailyReturn.append(closingPrice[i] / closingPrice[i - 1])
        continuousCompoundReturn.append(math.log(dailyReturn[i]))
    
    continuousCompoundReturnAvg = sum(continuousCompoundReturn) / len(closingPrice) # miu
    
    # calculate (LN(R) - miu) ^ 2
    thetaFangList = [0]
    for j in continuousCompoundReturn:
        thetaFangList.append(pow(j - continuousCompoundReturnAvg, 2))
    variance =  sum(thetaFangList) / (len(closingPrice) - 1) # 样本方差theta^2
    standardDeviation = pow(variance, 0.5)   #标准差，即波动率
    annualVolatility = standardDeviation * sqrt(tradingDaysPerYear)
    return annualVolatility

In [33]:
# def calculateStockPriceVolatilityGarch(stockCode, startDate, endDate, tradingDaysPerYear):
#     """
#     使用GARCH模型 计算标的在一段时间内的年化波动率
#     stockCode: 标的代码
#     startDate: 开始日期
#     endDate: 结束日期
#     tradingDaysPerYear: 一年交易日天数
#     """
#     data = w.wsd(stockCode, "pct_chg", startDate, endDate, "").Data
#     garch = arch_model(y = data, mean = 'Constant', lags = 0, vol = 'GARCH', p = 1, o = 0, q = 1, dist = 'normal')
#     garchmodel = garch.fit()
#     garchmodel.params

In [34]:
def calculate_S_T(stockClosingPrice, stockPriceVolatility, riskFreeRate, daysToExpire):
    """模拟epsilon，计算S_T
    stockClosingPrice：测试当日的标的价格
    """
    return stockClosingPrice * exp((riskFreeRate - 0.5 * stockPriceVolatility ** 2) * daysToExpire + stockPriceVolatility * sqrt(daysToExpire) * gauss(0.0, 1.0))

In [35]:
def option_payoff(flag, S_T, strkingPrice):
    """期权收益"""
    if flag.lower() == 'call':
        return max(S_T - strkingPrice, 0.0)
    else:
        return max(strkingPrice - S_T, 0.0)

In [36]:
def option_price(flag, stockClosingPrice, riskFreeRate, daysToExpire, stockPriceVolatility, strkingPrice, simulations):
    """模拟
    flag： call/put
    stockClosingPrice：测试日当日的标的收盘价
    riskFreeRate: 无风险利率
    daysToExpire：期权距离到期的天数
    stockPriceVolatility: 波动率
    strkingPrice: 期权行权价
    simulations: 蒙特卡洛模拟次数
    """
    payoffs = []
    discout = exp(-riskFreeRate * daysToExpire)
    for i in range(simulations):
        S_T = calculate_S_T(stockClosingPrice, stockPriceVolatility, riskFreeRate, daysToExpire)
        payoffs.append(
            option_payoff(flag, S_T, strkingPrice)
        )
    price = discout * sum(payoffs) / float(simulations)
    return price

In [37]:
def getOnePredict(stockCode, firstClosingPrice, dateList, riskFreeRate, tradingDaysPerYear, stockVolatility):
    """
    对于某股票/指数，获取其在某一段时间每一天的预测价格
    stockCode：股票/指数代码
    firstClosingPrice：要预测的时间段，第一天的收盘价，取自wind
    dateList：要预测的时间段（包含上述的第一天）
    riskFreeRate：无风险利率
    tradingDaysPerYear：一年交易日天数
    """
    result = [firstClosingPrice]
    previousDayClosingPrice = firstClosingPrice
#     volatility = calculateStockPriceVolatility(stockCode, dateList[0] + datetime.timedelta(days = -365), 
#                                                          dateList[0] + datetime.timedelta(days = -1), tradingDaysPerYear)
    for dateIndex in range(1, len(dateList)):
        testingDate_dt = dateList[dateIndex]

        predictedPrice = calculate_S_T(previousDayClosingPrice, stockVolatility, riskFreeRate, 1 / 365)
        result.append(predictedPrice)
        previousDayClosingPrice = predictedPrice
    return result

In [38]:
def drawLineChart(data, dates, optionName):
    #解决中文显示问题
    plt.rcParams['font.sans-serif']=['SimHei']
    plt.rcParams['axes.unicode_minus'] = False

    #指定像素
    plt.figure(dpi = 200)
    
    plt.xlabel('日期')
    plt.ylabel('预测价格')
    plt.title(optionName + '期权标的价格预测')
    plt.xticks(rotation = 30)   
    

    for i in range(0, len(data)):
        plt.plot(dates, data[i], linewidth = 0.5)
        print("第" + str(i + 1) + "条线已画")
    
    #去掉上轴和右轴
    axes = plt.gca()
    axes.spines['right'].set_color('none')
    axes.spines['top'].set_color('none')

    #设置x轴数据显示范围
    axes.set_xlim(dates[0] + datetime.timedelta(days = -1), dates[-1])
    #设置

In [39]:
def calculateOptionEndDateAvgPrice(simulationData, barrierPrice, secondLevelRR, participationRate, nominalAmount, duration):
    numberOfNonKnockOut = 0
    numberOfKnockOut = 0
    initialPrice = simulationData[0][0]
    knockOutTotalReturn = 0
    nonKnockOutTotalReturn = 0
    firstLevelReturns = []
    
    for data in simulationData:
        #未敲出，收益率按一级收益率计算
        if(data[-1] <= barrierPrice):
            firstLevelRateOfReturn = participationRate * max((data[-1] - initialPrice) / initialPrice, 0)
            firstLevelReturns.append(firstLevelRateOfReturn)
#             print("期末价格：" + str(data[-1]) + " 期初价格： " + str(initialPrice) + " 收益率：" + str(firstLevelRateOfReturn))
            returnPayment = nominalAmount * firstLevelRateOfReturn * duration / 365
            nonKnockOutTotalReturn += returnPayment
            numberOfNonKnockOut += 1    
        #敲出，收益按二级收益率计算
        else:
            numberOfKnockOut += 1
            returnPayment = nominalAmount * secondLevelRR * duration / 365
            knockOutTotalReturn += returnPayment
    
    #计算敲出与未敲出的概率
    nonKnockOutWeight = numberOfNonKnockOut / len(simulationData)
    knockOutWeight = 1 - nonKnockOutWeight
    
    #计算所有敲出情况的平均期末收益
    avgPeriodEndReturnPayment_knockOut = 0 if numberOfKnockOut == 0 else (knockOutTotalReturn / numberOfKnockOut)

    #计算所有未敲出情况的平均期末收益
    avgPeriodEndReturnPayment_nonKnockOut = 0 if numberOfNonKnockOut == 0 else (nonKnockOutTotalReturn / numberOfNonKnockOut)

    #计算所有预测的平均期末收益
    avgPeriodEndTotalReturnPayment = avgPeriodEndReturnPayment_knockOut * knockOutWeight + avgPeriodEndReturnPayment_nonKnockOut * nonKnockOutWeight
        
    nonKnockOutAvgRR = 0 if numberOfNonKnockOut == 0 else mean(firstLevelReturns)
    knockOutAvgRR = 0 if numberOfKnockOut == 0 else secondLevelRR
    
    return {"期权期末平均收益": avgPeriodEndTotalReturnPayment, "敲出概率": knockOutWeight, "未敲出概率": nonKnockOutWeight,
           "未敲出期末平均收益": avgPeriodEndReturnPayment_nonKnockOut, "敲出期末平均收益": avgPeriodEndReturnPayment_knockOut,
           "未敲出平均收益率": nonKnockOutAvgRR, "敲出平均收益率": knockOutAvgRR}

In [40]:

"""主程序"""

"""
测试的期权信息：
沪深300指数期权2011C4200
上市日：2020-08-24
到期日：2020-11-20
行权价：4200

对8.24 - 11.20的期权价格进行定价
"""

"""#################################################################"""
"""期权参数设置，需要手动填写"""

"""期权名称"""
OPTION_NAME = "中证500欧式看涨单鲨"

"""期权标的代码"""
STOCK_CODE = "000905.SH"

"""无风险利率 2020年国债一年期利率加权"""#http://www.chinamoney.com.cn/chinese/sddsintigy/
RISK_FREE_RATE = 2.1927 / 100

"""蒙特卡洛模拟次数"""
SIMULATIONS = 100000

"""一年交易日天数"""
TRADING_DATES_PER_YEAR = 240
DAYS_PER_YEAR = 365

"""期权开始和到期日"""
OPTION_START_DATE = "2020-06-23"
OPTION_STRIKE_DATE = "2021-03-15"

"""参与率"""
PARTICIPATION_RATE = 0.41

"""名义金额"""
AMOUNT = 69260000

"""执行价格"""
STRIKING_PRICE = 5804.34 

"""障碍价格"""
KNOCK_OUT_PRICE = 6674.99  

"""期权费率"""
PREMIUM_RATE = 0.0096 

"""二级收益率"""
SECOND_LEVEL_RATE_OF_RETURN = 0.0205

"""期权的行权方式，结果为认购/认沽"""
FLAG = "CALL"
"""#################################################################"""

"""获取期权交易天数（自然日）"""
DURATION = len(w.tdays(OPTION_START_DATE, OPTION_STRIKE_DATE, "Days=Alldays").Data[0])

"""计算期权本金"""
principal = AMOUNT * PREMIUM_RATE * DURATION / DAYS_PER_YEAR

"""获取期权交易日序列"""
dateList = getDatesList(OPTION_START_DATE, OPTION_STRIKE_DATE)#dateime格式

"""获取期权开始和到期日datetime格式"""
OPTION_START_DATE_dt = datetime.datetime.strptime(OPTION_START_DATE,'%Y-%m-%d')
OPTION_STRIKE_DATE_dt = datetime.datetime.strptime(OPTION_STRIKE_DATE,'%Y-%m-%d')
    
"""计算历史波动率"""
stockVolatility = calculateStockPriceVolatility(STOCK_CODE, OPTION_START_DATE_dt + datetime.timedelta(days = -365), 
                                                OPTION_START_DATE_dt + datetime.timedelta(days = -1), TRADING_DATES_PER_YEAR)
# stockVolatility = 0.014544779822168086 * sqrt(240)
print("波动率： " + str(stockVolatility))
results = []
firstDayStockClosingPrice = w.wsd(STOCK_CODE, "close", OPTION_START_DATE, OPTION_START_DATE, "").Data[0][0]

"""重复多次，获取N条期权标的预测价格Path"""
for i in range(0, SIMULATIONS):
    result = getOnePredict(STOCK_CODE, firstDayStockClosingPrice, dateList, RISK_FREE_RATE, TRADING_DATES_PER_YEAR, stockVolatility)
    results.append(result)
print("预测结束")

波动率： 0.22775662588426826
预测结束


In [41]:
principal = 486320.95

optionPayOffResults = calculateOptionEndDateAvgPrice(results, KNOCK_OUT_PRICE, SECOND_LEVEL_RATE_OF_RETURN, PARTICIPATION_RATE, AMOUNT, DURATION)
#期权预测期末平均价值
optionPayOff = optionPayOffResults["期权期末平均收益"]
#期权预测平均收益率


discout = exp(-RISK_FREE_RATE * DURATION/365)
price = optionPayOff * (DURATION / 365) * discout


averageRR = optionPayOffResults["未敲出平均收益率"] * optionPayOffResults["未敲出概率"] + optionPayOffResults["敲出平均收益率"] * optionPayOffResults["敲出概率"]
result = {"蒙特卡洛模拟":["期权到期日时的预期平均收益","预测期权价格","敲出概率","未敲出概率","未敲出收益均值","敲出收益均值","未敲出平均收益率","敲出平均收益率","平均收益率" ],
          "结果":["%.2f" % optionPayOff, 
                "%.2f" % price, 
                '{:.2%}'.format(optionPayOffResults["敲出概率"]), 
                '{:.2%}'.format(optionPayOffResults["未敲出概率"]), 
                "%.2f" % optionPayOffResults["未敲出期末平均收益"], 
                "%.2f" % optionPayOffResults["敲出期末平均收益"], 
                '{:.2%}'.format(optionPayOffResults["未敲出平均收益率"]), 
                '{:.2%}'.format(optionPayOffResults["敲出平均收益率"]), 
                '{:.2%}'.format(averageRR)]}
result_df = pd.DataFrame(result)
result_df


,蒙特卡洛模拟,结果
0,期权到期日时的预期平均收益,626883.87
1,预测期权价格,449610.02
2,敲出概率,18.50%
3,未敲出概率,81.50%
4,未敲出收益均值,534281.78
5,敲出收益均值,1034725.42
6,未敲出平均收益率,1.06%
7,敲出平均收益率,2.05%
8,平均收益率,1.24%


In [ ]:
drawLineChart(results, dateList, OPTION_NAME)

第1条线已画
第2条线已画
第3条线已画
第4条线已画
第5条线已画
第6条线已画
第7条线已画
第8条线已画
第9条线已画
第10条线已画
第11条线已画
第12条线已画
第13条线已画
第14条线已画
第15条线已画
第16条线已画
第17条线已画
第18条线已画
第19条线已画
第20条线已画
第21条线已画
第22条线已画
第23条线已画
第24条线已画
第25条线已画
第26条线已画
第27条线已画
第28条线已画
第29条线已画
第30条线已画
第31条线已画
第32条线已画
第33条线已画
第34条线已画
第35条线已画
第36条线已画
第37条线已画
第38条线已画
第39条线已画
第40条线已画
第41条线已画
第42条线已画
第43条线已画
第44条线已画
第45条线已画
第46条线已画
第47条线已画
第48条线已画
第49条线已画
第50条线已画
第51条线已画
第52条线已画
第53条线已画
第54条线已画
第55条线已画
第56条线已画
第57条线已画
第58条线已画
第59条线已画
第60条线已画
第61条线已画
第62条线已画
第63条线已画
第64条线已画
第65条线已画
第66条线已画
第67条线已画
第68条线已画
第69条线已画
第70条线已画
第71条线已画
第72条线已画
第73条线已画
第74条线已画
第75条线已画
第76条线已画
第77条线已画
第78条线已画
第79条线已画
第80条线已画
第81条线已画
第82条线已画
第83条线已画
第84条线已画
第85条线已画
第86条线已画
第87条线已画
第88条线已画
第89条线已画
第90条线已画
第91条线已画
第92条线已画
第93条线已画
第94条线已画
第95条线已画
第96条线已画
第97条线已画
第98条线已画
第99条线已画
第100条线已画
第101条线已画
第102条线已画
第103条线已画
第104条线已画
第105条线已画
第106条线已画
第107条线已画
第108条线已画
第109条线已画
第110条线已画
第111条线已画
第112条线已画
第113条线已画
第114条线已画
第115条线已画
第116条线已画
第117条线已画
第118条线已画
第119条线已画
第120条线已画
第121条线已画
第122条线已画
第123条线已画
第

第927条线已画
第928条线已画
第929条线已画
第930条线已画
第931条线已画
第932条线已画
第933条线已画
第934条线已画
第935条线已画
第936条线已画
第937条线已画
第938条线已画
第939条线已画
第940条线已画
第941条线已画
第942条线已画
第943条线已画
第944条线已画
第945条线已画
第946条线已画
第947条线已画
第948条线已画
第949条线已画
第950条线已画
第951条线已画
第952条线已画
第953条线已画
第954条线已画
第955条线已画
第956条线已画
第957条线已画
第958条线已画
第959条线已画
第960条线已画
第961条线已画
第962条线已画
第963条线已画
第964条线已画
第965条线已画
第966条线已画
第967条线已画
第968条线已画
第969条线已画
第970条线已画
第971条线已画
第972条线已画
第973条线已画
第974条线已画
第975条线已画
第976条线已画
第977条线已画
第978条线已画
第979条线已画
第980条线已画
第981条线已画
第982条线已画
第983条线已画
第984条线已画
第985条线已画
第986条线已画
第987条线已画
第988条线已画
第989条线已画
第990条线已画
第991条线已画
第992条线已画
第993条线已画
第994条线已画
第995条线已画
第996条线已画
第997条线已画
第998条线已画
第999条线已画
第1000条线已画
第1001条线已画
第1002条线已画
第1003条线已画
第1004条线已画
第1005条线已画
第1006条线已画
第1007条线已画
第1008条线已画
第1009条线已画
第1010条线已画
第1011条线已画
第1012条线已画
第1013条线已画
第1014条线已画
第1015条线已画
第1016条线已画
第1017条线已画
第1018条线已画
第1019条线已画
第1020条线已画
第1021条线已画
第1022条线已画
第1023条线已画
第1024条线已画
第1025条线已画
第1026条线已画
第1027条线已画
第1028条线已画
第1029条线已画
第1030条线已画
第1031条线已画
第1032条线已画
第1033条线已画
第10

第1821条线已画
第1822条线已画
第1823条线已画
第1824条线已画
第1825条线已画
第1826条线已画
第1827条线已画
第1828条线已画
第1829条线已画
第1830条线已画
第1831条线已画
第1832条线已画
第1833条线已画
第1834条线已画
第1835条线已画
第1836条线已画
第1837条线已画
第1838条线已画
第1839条线已画
第1840条线已画
第1841条线已画
第1842条线已画
第1843条线已画
第1844条线已画
第1845条线已画
第1846条线已画
第1847条线已画
第1848条线已画
第1849条线已画
第1850条线已画
第1851条线已画
第1852条线已画
第1853条线已画
第1854条线已画
第1855条线已画
第1856条线已画
第1857条线已画
第1858条线已画
第1859条线已画
第1860条线已画
第1861条线已画
第1862条线已画
第1863条线已画
第1864条线已画
第1865条线已画
第1866条线已画
第1867条线已画
第1868条线已画
第1869条线已画
第1870条线已画
第1871条线已画
第1872条线已画
第1873条线已画
第1874条线已画
第1875条线已画
第1876条线已画
第1877条线已画
第1878条线已画
第1879条线已画
第1880条线已画
第1881条线已画
第1882条线已画
第1883条线已画
第1884条线已画
第1885条线已画
第1886条线已画
第1887条线已画
第1888条线已画
第1889条线已画
第1890条线已画
第1891条线已画
第1892条线已画
第1893条线已画
第1894条线已画
第1895条线已画
第1896条线已画
第1897条线已画
第1898条线已画
第1899条线已画
第1900条线已画
第1901条线已画
第1902条线已画
第1903条线已画
第1904条线已画
第1905条线已画
第1906条线已画
第1907条线已画
第1908条线已画
第1909条线已画
第1910条线已画
第1911条线已画
第1912条线已画
第1913条线已画
第1914条线已画
第1915条线已画
第1916条线已画
第1917条线已画
第1918条线已画
第1919条线已画
第1920条线已画


第2764条线已画
第2765条线已画
第2766条线已画
第2767条线已画
第2768条线已画
第2769条线已画
第2770条线已画
第2771条线已画
第2772条线已画
第2773条线已画
第2774条线已画
第2775条线已画
第2776条线已画
第2777条线已画
第2778条线已画
第2779条线已画
第2780条线已画
第2781条线已画
第2782条线已画
第2783条线已画
第2784条线已画
第2785条线已画
第2786条线已画
第2787条线已画
第2788条线已画
第2789条线已画
第2790条线已画
第2791条线已画
第2792条线已画
第2793条线已画
第2794条线已画
第2795条线已画
第2796条线已画
第2797条线已画
第2798条线已画
第2799条线已画
第2800条线已画
第2801条线已画
第2802条线已画
第2803条线已画
第2804条线已画
第2805条线已画
第2806条线已画
第2807条线已画
第2808条线已画
第2809条线已画
第2810条线已画
第2811条线已画
第2812条线已画
第2813条线已画
第2814条线已画
第2815条线已画
第2816条线已画
第2817条线已画
第2818条线已画
第2819条线已画
第2820条线已画
第2821条线已画
第2822条线已画
第2823条线已画
第2824条线已画
第2825条线已画
第2826条线已画
第2827条线已画
第2828条线已画
第2829条线已画
第2830条线已画
第2831条线已画
第2832条线已画
第2833条线已画
第2834条线已画
第2835条线已画
第2836条线已画
第2837条线已画
第2838条线已画
第2839条线已画
第2840条线已画
第2841条线已画
第2842条线已画
第2843条线已画
第2844条线已画
第2845条线已画
第2846条线已画
第2847条线已画
第2848条线已画
第2849条线已画
第2850条线已画
第2851条线已画
第2852条线已画
第2853条线已画
第2854条线已画
第2855条线已画
第2856条线已画
第2857条线已画
第2858条线已画
第2859条线已画
第2860条线已画
第2861条线已画
第2862条线已画
第2863条线已画


第3635条线已画
第3636条线已画
第3637条线已画
第3638条线已画
第3639条线已画
第3640条线已画
第3641条线已画
第3642条线已画
第3643条线已画
第3644条线已画
第3645条线已画
第3646条线已画
第3647条线已画
第3648条线已画
第3649条线已画
第3650条线已画
第3651条线已画
第3652条线已画
第3653条线已画
第3654条线已画
第3655条线已画
第3656条线已画
第3657条线已画
第3658条线已画
第3659条线已画
第3660条线已画
第3661条线已画
第3662条线已画
第3663条线已画
第3664条线已画
第3665条线已画
第3666条线已画
第3667条线已画
第3668条线已画
第3669条线已画
第3670条线已画
第3671条线已画
第3672条线已画
第3673条线已画
第3674条线已画
第3675条线已画
第3676条线已画
第3677条线已画
第3678条线已画
第3679条线已画
第3680条线已画
第3681条线已画
第3682条线已画
第3683条线已画
第3684条线已画
第3685条线已画
第3686条线已画
第3687条线已画
第3688条线已画
第3689条线已画
第3690条线已画
第3691条线已画
第3692条线已画
第3693条线已画
第3694条线已画
第3695条线已画
第3696条线已画
第3697条线已画
第3698条线已画
第3699条线已画
第3700条线已画
第3701条线已画
第3702条线已画
第3703条线已画
第3704条线已画
第3705条线已画
第3706条线已画
第3707条线已画
第3708条线已画
第3709条线已画
第3710条线已画
第3711条线已画
第3712条线已画
第3713条线已画
第3714条线已画
第3715条线已画
第3716条线已画
第3717条线已画
第3718条线已画
第3719条线已画
第3720条线已画
第3721条线已画
第3722条线已画
第3723条线已画
第3724条线已画
第3725条线已画
第3726条线已画
第3727条线已画
第3728条线已画
第3729条线已画
第3730条线已画
第3731条线已画
第3732条线已画
第3733条线已画
第3734条线已画


...

第27766条线已画
第27767条线已画
第27768条线已画
第27769条线已画
第27770条线已画
第27771条线已画
第27772条线已画
第27773条线已画
第27774条线已画
第27775条线已画
第27776条线已画
第27777条线已画
第27778条线已画
第27779条线已画
第27780条线已画
第27781条线已画
第27782条线已画
第27783条线已画
第27784条线已画
第27785条线已画
第27786条线已画
第27787条线已画
第27788条线已画
第27789条线已画
第27790条线已画
第27791条线已画
第27792条线已画
第27793条线已画
第27794条线已画
第27795条线已画
第27796条线已画
第27797条线已画
第27798条线已画
第27799条线已画
第27800条线已画
第27801条线已画
第27802条线已画
第27803条线已画
第27804条线已画
第27805条线已画
第27806条线已画
第27807条线已画
第27808条线已画
第27809条线已画
第27810条线已画
第27811条线已画
第27812条线已画
第27813条线已画
第27814条线已画
第27815条线已画
第27816条线已画
第27817条线已画
第27818条线已画
第27819条线已画
第27820条线已画
第27821条线已画
第27822条线已画
第27823条线已画
第27824条线已画
第27825条线已画
第27826条线已画
第27827条线已画
第27828条线已画
第27829条线已画
第27830条线已画
第27831条线已画
第27832条线已画
第27833条线已画
第27834条线已画
第27835条线已画
第27836条线已画
第27837条线已画
第27838条线已画
第27839条线已画
第27840条线已画
第27841条线已画
第27842条线已画
第27843条线已画
第27844条线已画
第27845条线已画
第27846条线已画
第27847条线已画
第27848条线已画
第27849条线已画
第27850条线已画
第27851条线已画
第27852条线已画
第27853条线已画
第27854条线已画
第27855条线已画
第27856条线已画

第28517条线已画
第28518条线已画
第28519条线已画
第28520条线已画
第28521条线已画
第28522条线已画
第28523条线已画
第28524条线已画
第28525条线已画
第28526条线已画
第28527条线已画
第28528条线已画
第28529条线已画
第28530条线已画
第28531条线已画
第28532条线已画
第28533条线已画
第28534条线已画
第28535条线已画
第28536条线已画
第28537条线已画
第28538条线已画
第28539条线已画
第28540条线已画
第28541条线已画
第28542条线已画
第28543条线已画
第28544条线已画
第28545条线已画
第28546条线已画
第28547条线已画
第28548条线已画
第28549条线已画
第28550条线已画
第28551条线已画
第28552条线已画
第28553条线已画
第28554条线已画
第28555条线已画
第28556条线已画
第28557条线已画
第28558条线已画
第28559条线已画
第28560条线已画
第28561条线已画
第28562条线已画
第28563条线已画
第28564条线已画
第28565条线已画
第28566条线已画
第28567条线已画
第28568条线已画
第28569条线已画
第28570条线已画
第28571条线已画
第28572条线已画
第28573条线已画
第28574条线已画
第28575条线已画
第28576条线已画
第28577条线已画
第28578条线已画
第28579条线已画
第28580条线已画
第28581条线已画
第28582条线已画
第28583条线已画
第28584条线已画
第28585条线已画
第28586条线已画
第28587条线已画
第28588条线已画
第28589条线已画
第28590条线已画
第28591条线已画
第28592条线已画
第28593条线已画
第28594条线已画
第28595条线已画
第28596条线已画
第28597条线已画
第28598条线已画
第28599条线已画
第28600条线已画
第28601条线已画
第28602条线已画
第28603条线已画
第28604条线已画
第28605条线已画
第28606条线已画
第28607条线已画

第29262条线已画
第29263条线已画
第29264条线已画
第29265条线已画
第29266条线已画
第29267条线已画
第29268条线已画
第29269条线已画
第29270条线已画
第29271条线已画
第29272条线已画
第29273条线已画
第29274条线已画
第29275条线已画
第29276条线已画
第29277条线已画
第29278条线已画
第29279条线已画
第29280条线已画
第29281条线已画
第29282条线已画
第29283条线已画
第29284条线已画
第29285条线已画
第29286条线已画
第29287条线已画
第29288条线已画
第29289条线已画
第29290条线已画
第29291条线已画
第29292条线已画
第29293条线已画
第29294条线已画
第29295条线已画
第29296条线已画
第29297条线已画
第29298条线已画
第29299条线已画
第29300条线已画
第29301条线已画
第29302条线已画
第29303条线已画
第29304条线已画
第29305条线已画
第29306条线已画
第29307条线已画
第29308条线已画
第29309条线已画
第29310条线已画
第29311条线已画
第29312条线已画
第29313条线已画
第29314条线已画
第29315条线已画
第29316条线已画
第29317条线已画
第29318条线已画
第29319条线已画
第29320条线已画
第29321条线已画
第29322条线已画
第29323条线已画
第29324条线已画
第29325条线已画
第29326条线已画
第29327条线已画
第29328条线已画
第29329条线已画
第29330条线已画
第29331条线已画
第29332条线已画
第29333条线已画
第29334条线已画
第29335条线已画
第29336条线已画
第29337条线已画
第29338条线已画
第29339条线已画
第29340条线已画
第29341条线已画
第29342条线已画
第29343条线已画
第29344条线已画
第29345条线已画
第29346条线已画
第29347条线已画
第29348条线已画
第29349条线已画
第29350条线已画
第29351条线已画
第29352条线已画

第30012条线已画
第30013条线已画
第30014条线已画
第30015条线已画
第30016条线已画
第30017条线已画
第30018条线已画
第30019条线已画
第30020条线已画
第30021条线已画
第30022条线已画
第30023条线已画
第30024条线已画
第30025条线已画
第30026条线已画
第30027条线已画
第30028条线已画
第30029条线已画
第30030条线已画
第30031条线已画
第30032条线已画
第30033条线已画
第30034条线已画
第30035条线已画
第30036条线已画
第30037条线已画
第30038条线已画
第30039条线已画
第30040条线已画
第30041条线已画
第30042条线已画
第30043条线已画
第30044条线已画
第30045条线已画
第30046条线已画
第30047条线已画
第30048条线已画
第30049条线已画
第30050条线已画
第30051条线已画
第30052条线已画
第30053条线已画
第30054条线已画
第30055条线已画
第30056条线已画
第30057条线已画
第30058条线已画
第30059条线已画
第30060条线已画
第30061条线已画
第30062条线已画
第30063条线已画
第30064条线已画
第30065条线已画
第30066条线已画
第30067条线已画
第30068条线已画
第30069条线已画
第30070条线已画
第30071条线已画
第30072条线已画
第30073条线已画
第30074条线已画
第30075条线已画
第30076条线已画
第30077条线已画
第30078条线已画
第30079条线已画
第30080条线已画
第30081条线已画
第30082条线已画
第30083条线已画
第30084条线已画
第30085条线已画
第30086条线已画
第30087条线已画
第30088条线已画
第30089条线已画
第30090条线已画
第30091条线已画
第30092条线已画
第30093条线已画
第30094条线已画
第30095条线已画
第30096条线已画
第30097条线已画
第30098条线已画
第30099条线已画
第30100条线已画
第30101条线已画
第30102条线已画

KeyboardInterrupt: 

In [56]:
def getGreeks(stockPrice, strikingPrice, riskFreeRate, T, volatility, n = 1):
    """
    T: 距离到期日的天数/365
    volatility为标的资产的历史波动率。例如，计算50ETF期权的Greeks时，sigma常用过去60天50ETF对数收益率的标准差，再将其年化后来表示。
    """
    daltaResult = delta(stockPrice, strikingPrice, riskFreeRate, T, volatility, n)
    gammaResult = gamma(stockPrice, strikingPrice, riskFreeRate, T, volatility)
    vegaResult = vega(stockPrice, strikingPrice, riskFreeRate, T, volatility)
    thetaResult = theta(stockPrice, strikingPrice, riskFreeRate, T, volatility, n)
    print("delta: " + str(daltaResult))
    print("gamma: " + str(gammaResult))
    print("vega: " + str(vegaResult))
    print("theta: " + str(thetaResult))

In [43]:
def d(stockPrice, strikingPrice, riskFreeRate, T, volatility):
    d1 = (np.log(stockPrice / strikingPrice) + (riskFreeRate + 0.5 * volatility ** 2) * T) / (volatility * np.sqrt(T))
    d2 = d1 - volatility * np.sqrt(T)
    return (d1, d2)

In [44]:
def delta(stockPrice, strikingPrice, riskFreeRate, T, volatility, n):
    """
    认购期权的n为1
    认沽期权的n为-1
    """
    d1 = d(stockPrice, strikingPrice, riskFreeRate, T, volatility)[0]
    delta = n * si.norm.cdf(n * d1)
    return delta

In [45]:
def gamma(stockPrice, strikingPrice, riskFreeRate, T, volatility):
    d1 = d(stockPrice, strikingPrice, riskFreeRate, T, volatility)[0]
    gamma = si.norm.pdf(d1) / (s * volatility * np.sqrt(T))
    return gamma

In [46]:
def vega(stockPrice, strikingPrice, riskFreeRate, T, volatility):
    d1 = d(stockPrice, strikingPrice, riskFreeRate, T, volatility)[0]
    vega = (stockPrice * si.norm.pdf(d1) * np.sqrt(T)) / 100
    return vega

In [47]:
def theta(stockPrice, strikingPrice, riskFreeRate, T, volatility, n):
    '''
    认购期权的n为1
    认沽期权的n为-1
    '''
    d1 = d(stockPrice, strikingPrice, riskFreeRate, T, volatility)[0]
    d2 = d(stockPrice, strikingPrice, riskFreeRate, T, volatility)[1]
    
    theta = (-1 * (stockPrice * si.norm.pdf(d1) * volatility) / (2 * np.sqrt(T)) - n * riskFreeRate * strikingPrice * np.exp(-riskFreeRate * T) * si.norm.cdf(n * d2)) / 365
    return theta

In [57]:
s = getStockData(OPTION_START_DATE, OPTION_STRIKE_DATE, STOCK_CODE).Data[0][0]

getGreeks(s, STRIKING_PRICE, RISK_FREE_RATE, DURATION/365, stockVolatility, n = 1)

delta: 0.5711901596858417
gamma: 0.0003478583669705666
vega: 19.452184650300957
theta: -1.002299225327729


# 文字解释部分

以每股苹果股票为标的的期权定价例子：

https://zhuanlan.zhihu.com/p/134759744

用19.4.23 - 20.4.23的股价计算波动率，

谷歌得到一年期的无风险利率r=0.16%

找出苹果公司期权的最新报价。即还有58天到期的期权

距离行权日还有T年（如果是n“天”，那么把n化成n/365年），到期日标的资产价格为 S(T)，标的资产行权价格为K，那么在行权日T时刻，期权的价格为：

Call option： C(T) = max(0, S(T) - K)

Put option：  P(T) = max(0, K - S(T))

使用蒙卡模拟进行期权定价，需要在风险中性环境中（风险中性定价原则认为1、标的资产的预期收益率=无风险利率，2、从而对未来的现金流进行无风险利率折现

因此在第t日，期权的价格为距离到期日T的预期收益折现值

![](optionPricingEquation.png)

接下来，我们需要对股价S(T)进行模拟。在BS模型中，股价服从几何布朗运动，推导得到

![](optionPricingEquation2.png)




Reference:

Code and Finance 2 蒙特卡洛模拟期权定价 - 苹果公司例子 https://zhuanlan.zhihu.com/p/134759744

场外期权系列——鲨鱼鳍期权的蒙特卡洛定价 https://zhuanlan.zhihu.com/p/163862299?utm_source=wechat_session

鲨鱼鳍期权定价、风险对冲和理财产品 https://www.gfedu.cn/aqf/content_22650.shtml

兴业证券 - 鲨鱼鳍期权及其理财产品 http://www.doc88.com/p-9082540939123.html

场外期权系列——鲨鱼鳍期权的蒙特卡洛定价 https://www.jianshu.com/p/5d4bf6e7d426

沪深300期权价格
http://www.cffex.com.cn/hs300gzqq/

暨南大学论文：http://www.doc88.com/p-9965019376435.html

入门期权隐含波动率计算 https://zhuanlan.zhihu.com/p/139521048


https://zhuanlan.zhihu.com/p/24482145

In [16]:
# optionData = w.wss(OPTION_CODE, "startdate, exe_enddate, exe_price, exe_mode").Data #获取期权开始和到期日，行权价格
# OPTION_START_DATE_dt = optionData[0][0]
# OPTION_STRIKE_DATE_dt = optionData[1][0]
# optionData = w.wsd(STOCK_CODE, "close", OPTION_START_DATE_dt + datetime.timedelta(days = -365), OPTION_STRIKE_DATE_dt, "")
# optionData

In [47]:
# d1 = datetime.datetime.strptime("2020-01-01",'%Y-%m-%d')
# d2 = datetime.datetime.strptime("2020-01-08",'%Y-%m-%d')
# d3 = datetime.datetime.strptime("2020-01-09",'%Y-%m-%d')
# d4 = datetime.datetime.strptime("2020-01-10",'%Y-%m-%d')
# d5 = datetime.datetime.strptime("2020-01-13",'%Y-%m-%d')
# x = [d1, d2 , d3, d4, d5]
# y = [4, 3, 2, 1, 5]

# plt.plot(x,y,color='red',linewidth=2,linestyle='--')
# ax = plt.gca()
# ax.spines['right'].set_color('none')
# ax.spines['top'].set_color('none')
# # ax.spines['left'].set_position(('outward', -165))
# # ax.spines['bottom'].set_position(('outward', -5))
# plt.xticks(d1, d5, 2)

In [67]:
data = w.wsd("000905.SH", "pct_chg", "2019-06-22", "2020-06-22", "").Data[0]
data1 = []
for d in data:
    data1.append(d/100)

garch = arch_model(y = data1, mean = 'Constant', lags = 0, vol = 'GARCH', p = 1, o = 0, q = 1, dist = 'normal')
garchmodel = garch.fit()
# garchmodel.params
vol = np.sqrt(garchmodel.params[1] / (1 - garchmodel.params[2] - garchmodel.params[3]))
print("长期波动率: " + str(vol))

Iteration:      1,   Func. Count:      6,   Neg. LLF: -700.9408399010165
Iteration:      2,   Func. Count:     18,   Neg. LLF: -700.9870161487015
Iteration:      3,   Func. Count:     30,   Neg. LLF: -701.0098353690064
Iteration:      4,   Func. Count:     43,   Neg. LLF: -701.0162590362243
Iteration:      5,   Func. Count:     52,   Neg. LLF: -701.0231281977706
Iteration:      6,   Func. Count:     64,   Neg. LLF: -701.0278003723195
Iteration:      7,   Func. Count:     76,   Neg. LLF: -701.0291060736495
Iteration:      8,   Func. Count:     89,   Neg. LLF: -701.0299294356905
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -701.0299289409793
            Iterations: 11
            Function evaluations: 99
            Gradient evaluations: 8
长期波动率: 0.014544779822168086


C:\Users\wangsu\AppData\Local\Continuum\anaconda3\lib\site-packages\arch\univariate\base.py:293: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002114. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  data_scale_warning.format(orig_scale, rescale), DataScaleWarning


In [23]:
#可以看到GARCH(1,1)模型的参数ω=0.00000423，α=0.101574，β=0.878452
a = 2
nonKnockOutAvgRR = 0 if a == 0 else 1
nonKnockOutAvgRR

1